In [ ]:
%load_ext autoreload
%autoreload 2

from trading import api

trades_unit = api.fetch_data()
trades_date = api.convert_to_time(trades_unit)
trades_unit = api.compute_stats(trades_unit)
trades_date = api.compute_stats(trades_date.reset_index(), all_stats=False).set_index(('Date', '.'))

# print(trades_unit.columns)
# print(trades_unit.head())
# print(trades_date.columns)
# print(trades_date.head())

In [ ]:
import plotly.graph_objects as go
trades = trades_unit.copy()
periods = ['Globally', 'Yearly', 'Monthly', 'Weekly']

fig = go.Figure(go.Bar(x=trades.index.to_list(), y=trades[('Gain', 'Globally')].to_list(), name='Trades'))

for col in trades['Balance'].columns.get_level_values(0):
    for period in periods:
        if period in col:
            fig.add_trace(go.Scatter(x=trades.index.to_list(), y=trades[('Balance', col)].to_list(), name='Balance '+col.replace('_', ' ')))

fig.show()

In [ ]:
import pandas as pd
import numpy as np
trades = trades_unit.copy().ffill()
PERIOD = ['Globally', 'Yearly', 'Monthly', 'Weekly']

stats = pd.DataFrame()
for metric, period in trades.columns:
    for p in ['Globally', 'Yearly', 'Monthly', 'Weekly']:
        if p in period and metric in ['Count', 'Balance', 'CountTP', 'RateTP', 'PayoffTP', 'CountBE', 'RateBE', 'PayoffBE', 'CountSL', 'RateSL', 'PayoffSL']:
            stats.loc[period, metric] = trades[(metric, period)].iloc[-1]
stats['WinRate'] = stats['RateTP']
stats['PayoffRatio'] = stats['PayoffTP'] / -stats['PayoffSL']
stats

In [ ]:
gain = [col for col in trades.columns if 'Gain' in col] 
balance = [col for col in trades.columns if 'Balance' in col] 
trades_date = pd.concat([
    pd.DataFrame({'Date': pd.date_range(start=trades['Date'].min(), end=trades['Date'].max())}).set_index('Date'),
    trades.set_index('Date').groupby('Date')[gain].sum(),
    trades.set_index('Date').groupby('Date')[balance].last(),
], axis=1)
trades_date[[col for col in trades.columns if 'Balance_' in col]].head(50)

In [97]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from trading import api

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
trades = api.fetch_data()
print(trades.columns)


    # ------------------------------------
    # Connect to Google Sheet
    # and fetch trades data
    # for Trading Dashboard 
    # ------------------------------------
    
Index(['Date', 'Gain', 'Result', 'Risque'], dtype='object', name=0)


In [101]:
import plotly.graph_objects as go
trades_graph = trades.copy()
# trades_graph['Gain'] = trades_graph['Gain']/10000*100
trades_graph['Gain'] = trades_graph['Gain']/trades_graph['Risque']
trades_graph = api.compute_stats(trades_graph)
periods = ['Globally', 'Yearly']

fig = go.Figure(go.Bar(x=trades_graph.index.to_list(), y=trades_graph[('Gain', 'Globally')].to_list(), name='Trades'))
for col in trades_graph['Balance'].columns.get_level_values(0):
    for period in periods:
        if period in col:
            fig.add_trace(go.Scatter(x=trades_graph.index.to_list(), y=trades_graph[('Balance', col)].to_list(), name=col.replace('_', ' ')))

# fig.update_yaxes(tickformat = '.1f', ticksuffix=' %', dtick = 0.5)
# fig.update_yaxes(tickformat = '.0f', ticksuffix=' €', dtick = 50)
fig.update_yaxes(tickformat = '.0f', ticksuffix=' RR', dtick = 1)
fig.show()

In [64]:
trades_stats = api.compute_stats(trades)
periods = ['Globally', 'Yearly']
periods = ['Globally', 'Yearly', 'Monthly', 'Weekly']


s = trades_stats.ffill().iloc[-1]
# for idx in s.index:
#     print(idx, s[idx])

METRICS = [m+r for r in ['TP', 'BE', 'SL'] for m in ['Count', 'Rate', 'Payoff']]
stats = pd.DataFrame(columns=['Balance', 'Count', 'WinRate', 'PayoffRatio', *METRICS])
# print(stats)

for metric, period in s.index:
    if metric in stats.columns and period.split('_')[0] in periods:
        stats.loc[period, metric] = s[(metric, period)]

stats['WinRate'] = stats['RateTP']
stats['PayoffRatio'] = stats['PayoffTP'] / -stats['PayoffSL']


stats.iloc[::-1].style \
.format({
    "Balance": "{:.0f} €",
    "Count": "{:.0f}",
    "WinRate": "{:.0%}",
    "PayoffRatio": "{:.2f}",
    "CountTP": "{:.0f}",
    "RateTP": "{:.0%}",
    "PayoffTP": "{:.0f} €",
    "CountBE": "{:.0f}",
    "RateBE": "{:.0%}",
    "PayoffBE": "{:.0f} €",
    "CountSL": "{:.0f}",
    "RateSL": "{:.0%}",
    "PayoffSL": "{:.0f} €",
}) \
.set_properties(**{'font-weight': 'bold'}, subset=['Balance']) \
.highlight_null(props="color: transparent;")

,Balance,Count,WinRate,PayoffRatio,CountTP,RateTP,PayoffTP,CountBE,RateBE,PayoffBE,CountSL,RateSL,PayoffSL
Weekly_2023-14,-52 €,3,nan%,nan,nan,nan%,nan €,nan,nan%,nan €,3,100%,-17 €
Monthly_2023-4,-52 €,3,nan%,nan,nan,nan%,nan €,nan,nan%,nan €,3,100%,-17 €
Weekly_2023-13,-24 €,1,nan%,nan,nan,nan%,nan €,nan,nan%,nan €,1,100%,-24 €
Monthly_2023-3,-24 €,1,nan%,nan,nan,nan%,nan €,nan,nan%,nan €,1,100%,-24 €
Weekly_2023-6,23 €,2,50%,2.00,1,50%,47 €,nan,nan%,nan €,1,100%,-23 €
Monthly_2023-2,23 €,2,50%,2.00,1,50%,47 €,nan,nan%,nan €,1,100%,-23 €
Weekly_2023-4,-13 €,1,nan%,nan,nan,nan%,nan €,nan,nan%,nan €,1,100%,-13 €
Weekly_2023-3,-2 €,2,nan%,nan,nan,nan%,nan €,2,100%,-1 €,nan,nan%,nan €
Weekly_2023-2,-22 €,1,nan%,nan,nan,nan%,nan €,nan,nan%,nan €,1,100%,-22 €
Monthly_2023-1,-37 €,4,nan%,nan,nan,nan%,nan €,2,67%,-1 €,2,50%,-17 €
